# 03 - Grid / Block Model
Proposito: generar la grilla de bloques desde config o extents.

Inputs:
- `configs/config.yml`
- CSV en `cfg["data_csv_path"]`

Outputs esperados:
- `outputs/tables/grid.csv`


### 1. Project setup


In [1]:
import os, sys, json, glob
from geostats_pipeline.config import load_config
from IPython.display import Image, display

PROJECT_ROOT = os.path.abspath(os.path.join(os.getcwd(), ".."))
if os.path.basename(os.getcwd()) == "notebooks":
    os.chdir(PROJECT_ROOT)
else:
    PROJECT_ROOT = os.getcwd()
    os.chdir(PROJECT_ROOT)

if PROJECT_ROOT not in sys.path:
    sys.path.insert(0, PROJECT_ROOT)

print("PROJECT_ROOT:", PROJECT_ROOT)
print("CWD:", os.getcwd())

from src.preprocess import load_and_preprocess
from src.grid import grid_from_extents, make_grid_dataframe, export_grid_to_csv


PROJECT_ROOT: c:\Users\joelm\Documents\geostats
CWD: c:\Users\joelm\Documents\geostats


### 2. Load config


In [2]:
cfg_path = "configs/config.yml"
print("Config:", os.path.abspath(cfg_path))
cfg = load_config(cfg_path)

print("Data CSV:", cfg["data_csv_path"])
mapping = {
    "x": cfg["columns"].get("x"),
    "y": cfg["columns"].get("y"),
    "z": cfg["columns"].get("z"),
    "var": cfg["columns"].get("variable_objetivo"),
}
print("Column mapping:", mapping)


Config: c:\Users\joelm\Documents\geostats\config\project.json
Data CSV: csv/Conminution.csv
Column mapping: {'x': 'X', 'y': 'Y', 'z': 'Z', 'var': 'Bwi_kWh_t'}


### 3. Load data


In [3]:
df, df_raw, mapping = load_and_preprocess(cfg)

rows_before = len(df_raw)
rows_after = len(df)
removed_pct = 0.0 if rows_before == 0 else (rows_before - rows_after) * 100.0 / rows_before

print("Shape raw:", df_raw.shape, "clean:", df.shape)
print("Dtypes:\n", df.dtypes)
print("Rows removed (%):", f"{removed_pct:.2f}")
print("X range:", (df["x"].min(), df["x"].max()))
print("Y range:", (df["y"].min(), df["y"].max()))
if "z" in df.columns and df["z"].notna().any():
    print("Z range:", (df["z"].min(), df["z"].max()))
else:
    print("Z range: n/a")


Shape raw: (600, 12) clean: (600, 12)
Dtypes:
 x             float64
y             float64
z             float64
Hole ID           str
Samples_ID        str
Lote              str
domain            str
Minz              str
EM_new            str
EM                str
var           float64
Axb_SMC       float64
dtype: object
Rows removed (%): 0.00
X range: (np.float64(2363710.832), np.float64(2365405.149))
Y range: (np.float64(6485685.572631), np.float64(6487146.812069))
Z range: (np.float64(3104.889936), np.float64(3834.876096))


### 4. Build grid


In [4]:
import math

grid_cfg = cfg.get("grid", {})
dx = float(grid_cfg.get("dx"))
dy = float(grid_cfg.get("dy"))
dz = float(grid_cfg.get("dz"))

padding_x = float(grid_cfg.get("padding_x", 0.0))
padding_y = float(grid_cfg.get("padding_y", 0.0))
padding_z = float(grid_cfg.get("padding_z", 0.0))

if dx <= 0 or dy <= 0 or dz <= 0:
    raise ValueError("dx, dy, dz must be positive")

xmin = float(df["x"].min())
xmax = float(df["x"].max())
ymin = float(df["y"].min())
ymax = float(df["y"].max())
if "z" in df.columns and df["z"].notna().any():
    zmin = float(df["z"].min())
    zmax = float(df["z"].max())
else:
    zmin, zmax = 0.0, 0.0

data_extents = {
    "xmin": xmin,
    "xmax": xmax,
    "ymin": ymin,
    "ymax": ymax,
    "zmin": zmin,
    "zmax": zmax,
}

xmin -= padding_x
xmax += padding_x
ymin -= padding_y
ymax += padding_y
zmin -= padding_z
zmax += padding_z

nx = int(math.ceil((xmax - xmin) / dx))
ny = int(math.ceil((ymax - ymin) / dy))
if "z" in df.columns and df["z"].notna().any():
    nz = int(math.ceil((zmax - zmin) / dz))
else:
    nz = 1

grid_spec = {
    "nx": max(nx, 1),
    "ny": max(ny, 1),
    "nz": max(nz, 1),
    "xmin": float(xmin),
    "ymin": float(ymin),
    "zmin": float(zmin),
    "dx": float(dx),
    "dy": float(dy),
    "dz": float(dz),
}

grid_df = make_grid_dataframe(grid_spec)
os.makedirs("outputs/tables", exist_ok=True)
export_grid_to_csv(grid_df, "outputs/tables/grid.csv")

total_blocks = int(grid_spec["nx"] * grid_spec["ny"] * grid_spec["nz"])
print("Data extents (raw):", data_extents)
print("Block size (m):", {"dx": dx, "dy": dy, "dz": dz})
print("Grid origin (xmin,ymin,zmin):", (grid_spec["xmin"], grid_spec["ymin"], grid_spec["zmin"]))
print("Grid dims (nx,ny,nz):", (grid_spec["nx"], grid_spec["ny"], grid_spec["nz"]))
print("Total blocks:", total_blocks)

print(grid_df.head())


Data extents (raw): {'xmin': 2363710.832, 'xmax': 2365405.149, 'ymin': 6485685.572631, 'ymax': 6487146.812069, 'zmin': 3104.889936, 'zmax': 3834.876096}
Block size (m): {'dx': 25.0, 'dy': 25.0, 'dz': 5.0}
Grid origin (xmin,ymin,zmin): (2363710.832, 6485685.572631, 3104.889936)
Grid dims (nx,ny,nz): (68, 59, 146)
Total blocks: 585752
             x             y            z
0  2363723.332  6.485698e+06  3107.389936
1  2363723.332  6.485698e+06  3112.389936
2  2363723.332  6.485698e+06  3117.389936
3  2363723.332  6.485698e+06  3122.389936
4  2363723.332  6.485698e+06  3127.389936


### 5. Artifacts generated


In [5]:
expected_figures = []
expected_tables = ['outputs/tables/grid.csv']
expected_models = []

def _existing(paths):
    return [p for p in paths if os.path.exists(p)]

figure_paths = _existing(expected_figures)
table_paths = _existing(expected_tables)
model_paths = _existing(expected_models)

print("Figures:", [os.path.abspath(p) for p in figure_paths])
print("Tables:", [os.path.abspath(p) for p in table_paths])
print("Models:", [os.path.abspath(p) for p in model_paths])

for p in figure_paths:
    display(Image(filename=p))


Figures: []
Tables: ['c:\\Users\\joelm\\Documents\\geostats\\outputs\\tables\\grid.csv']
Models: []
